<a href="https://colab.research.google.com/github/spaceml-org/Curator-Unlabeled-Image-Search-Guide/blob/main/notebooks/Active_Labeler.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This demo will demonstrate how to use a pretrained self-supervised model with the active labeler's command-line interface. We used [RESISC45](https://www.tensorflow.org/datasets/catalog/resisc45) in this demo. Although RESISC45 has labels, we first set up the dataset as an unlabeled dataset to demonstrate how to use unlabeled datasets in this pipeline.




# 1. Setup

## 1.1 Installing packages & Active Labeler

In [ ]:
%cd "/content"
import os
import shutil
if os.path.exists('/content/Active-Labeler'):
  shutil.rmtree('/content/Active-Labeler')

!git clone https://github.com/spaceml-org/Active-Labeler.git

In [ ]:
!pip install -r /content/Active-Labeler/requirements.txt

In [ ]:
!pip install --extra-index-url https://developer.download.nvidia.com/compute/redist nvidia-dali-cuda100

## 1.2 Setting up dataset and SSL model

In [ ]:
import os
import shutil
import pathlib
from pathlib import Path
from imutils import paths

In [ ]:
if os.path.exists("/content/RESISC45"):
  shutil.rmtree("/content/RESISC45")

!gdown https://drive.google.com/uc?id=14zEhqi9mczZaLEb33TQuKbhmurn2ClGL&export=download
!unrar x /content/RESISC45.rar
!mv NWPU-RESISC45/ RESISC45/
!rm -rf /content/RESISC45.rar
 
folder = '/content/Dataset/Unlabeled'
if os.path.exists(folder):
    shutil.rmtree(folder)

pathlib.Path(folder).mkdir(parents=True, exist_ok=True)

for i in paths.list_images('/content/RESISC45'):
  shutil.copy(i,os.path.join(folder,i.split('/')[-1]))

In [ ]:
from imutils import paths
list(paths.list_images("/content/runtime/swipe/archive"))

In [ ]:
!gdown https://drive.google.com/uc?id=1h2rm2SrcsqBXoxoHqzseuKtQfX3EVj-8

# 2. Active Labeler CLI Tool

## 2-1. Changing config files

The code cell below will change *model_config.yaml* file and *pipeline_config.yml* file so that the CLI tool runs on Colab. If you want to run it on your local device, you will have to manually change the config files. The main changes are the location of the SSL model and the refrence image along with model's embedding size. 

In [ ]:
# Edit the config files
import yaml

with open("/content/Active-Labeler/model_config.yaml") as f:
     list_doc = yaml.safe_load(f)

list_doc["encoder"]["encoder_path"] = "/content/RESISC45-imagenet_resnet18.ckpt"
list_doc["encoder"]["e_embedding_size"] = 512

with open("/content/Active-Labeler/model_config.yaml", "w") as f:
    yaml.dump(list_doc, f, default_flow_style=False)


with open("/content/Active-Labeler/pipeline_config.yaml") as f:
     list_doc = yaml.safe_load(f)

list_doc["model"]["model_path"] = "/content/RESISC45-imagenet_resnet18.ckpt"
list_doc["model"]["embedding_size"] = 512
list_doc["seed_dataset"]["ref_img_path"] = "/content/RESISC45/airplane/airplane_001.jpg"

with open("/content/Active-Labeler/pipeline_config.yaml", "w") as f:
    yaml.dump(list_doc, f, default_flow_style=False)

## 2-2. Running Active Labeler

Once you run the cell below, you'll get a URL for labeling images. 

**Note**: This link only works on Colab. If you are running the CLI tool on your local device, you will get a different link (http://0.0.0.0:5000/) after you run the CLI command. Use that link for your local device.

In [ ]:
from google.colab.output import eval_js
print(eval_js("google.colab.kernel.proxyPort(5000)"))

In [ ]:
!python3 /content/Active-Labeler/main.py --config_path /content/Active-Labeler/pipeline_config.yaml